In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

from datetime import datetime, timedelta
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows

from feature.scalers import ranged_scaler
from feature.engineering import *
from CARAT.model_utils import *
from CARAT.model import CausalGraphVAE
from CARAT.components import *
from utils.utils import set_seed, logger

import torch
import random
import numpy as np

# Define the seed value
seed = 42

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA (if using GPUs)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# Set seed for Python's random module
random.seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Ensure deterministic behavior for PyTorch operations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Set device
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':16:8'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = "1"

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device( "cpu")
# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
print(cats_df.shape)
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

(5000000, 20)


In [3]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [4]:
cats_rows_list = metadata.rows(named=True)
cats_df.shape

(5000000, 21)

In [5]:
cats_df = make_stationary(cats_df)

⚠️ Skipping column timestamp (not a float column)
Column: aimp | Final ADF: -2211.6294 | p-value: 0.0000 | Diffs: 0
Column: amud | Final ADF: -8.0091 | p-value: 0.0000 | Diffs: 0
Column: arnd | Final ADF: -18.1248 | p-value: 0.0000 | Diffs: 0
Column: asin1 | Final ADF: -5.7565 | p-value: 0.0000 | Diffs: 1
Column: asin2 | Final ADF: -39.8976 | p-value: 0.0000 | Diffs: 3
Column: adbr | Final ADF: -111.5305 | p-value: 0.0000 | Diffs: 0
Column: adfl | Final ADF: -217.2031 | p-value: 0.0000 | Diffs: 0
Column: bed1 | Final ADF: -35.0214 | p-value: 0.0000 | Diffs: 0
Column: bed2 | Final ADF: -44.5328 | p-value: 0.0000 | Diffs: 0
Column: bfo1 | Final ADF: -8.5614 | p-value: 0.0000 | Diffs: 0
Column: bfo2 | Final ADF: -13.3015 | p-value: 0.0000 | Diffs: 0
Column: bso1 | Final ADF: -2164.1026 | p-value: 0.0000 | Diffs: 1
Column: bso2 | Final ADF: -13.5707 | p-value: 0.0000 | Diffs: 0
Column: bso3 | Final ADF: -14.7270 | p-value: 0.0000 | Diffs: 0
Column: ced1 | Final ADF: -727.3289 | p-value: 0.

In [6]:
cats_df = cats_df.to_pandas()

In [7]:
metadata['affected'].unique().to_list()

["['cfo1']", "['ced1']", "['cso1']"]

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:06,0.0,0.142857,-0.444480,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000072,-0.507953,-0.716059,0.0,0.100401,-0.186461
2023-01-01 00:00:07,0.0,0.142857,-0.446078,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000083,-0.507953,-0.716059,0.0,0.100408,-0.186406
2023-01-01 00:00:08,0.0,0.142857,-0.447166,0.00002,-8.000267e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000094,-0.507953,-0.716059,0.0,0.100416,-0.186345
2023-01-01 00:00:09,0.0,0.142857,-0.442843,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000104,-0.507953,-0.716059,0.0,0.100426,-0.186278
2023-01-01 00:00:10,0.0,0.142857,-0.441320,0.00002,-8.000045e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000113,-0.507953,-0.716059,0.0,0.100438,-0.186205


In [10]:
cats_df.shape

(4999994, 17)

In [11]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [12]:
test_df = test_df.astype('float32')

In [13]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [14]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

In [15]:
import torch
import random
import numpy as np

# Define the seed value
seed = 42

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA (if using GPUs)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

# Set seed for Python's random module
random.seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Ensure deterministic behavior for PyTorch operations
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
TIME_STEPS = 15
BATCH_SIZE = 10000
hidden_dim = 64
latent_dim = 16
num_nodes = 17

dataset_nominal = TimeSeriesDataset(train_df, device=device, time_steps=TIME_STEPS)
dataloader_nominal = DataLoader(dataset_nominal, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)

# Initialize model and optimizer
model = CausalGraphVAE(input_dim=train_df.shape[1], hidden_dim=hidden_dim,
                        latent_dim=latent_dim, num_nodes=train_df.shape[1],device=device,
                        time_steps=TIME_STEPS, prior_adj=None,instantaneous_weight=0.4)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Train on nominal data
print("Pretraining on nominal data...")
model.train_model(dataloader_nominal, optimizer, num_epochs=250, patience=25,rho_max=3,alpha_max=1.5)


Pretraining on nominal data...
Epoch 1: Loss = 184355.7912
Recon Loss = 50394.9375, KL Loss = 0.0000, Lagrangian Loss = 19456.3281
Epoch 51: Loss = 749.3755
Recon Loss = 717.3883, KL Loss = 3.6515, Lagrangian Loss = 13.9407
Epoch 101: Loss = 283.9353
Recon Loss = 281.1418, KL Loss = 5.7118, Lagrangian Loss = 8.6988
Epoch 151: Loss = 205.1650
Recon Loss = 176.3573, KL Loss = 5.5914, Lagrangian Loss = 10.7432
Epoch 201: Loss = 184.4385
Recon Loss = 186.6451, KL Loss = 5.4285, Lagrangian Loss = 13.7190


In [17]:
prior_adj = model.causal_graph.adj_mat.clone().detach()
prior_adj

tensor([[0.0000, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.0000, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.2277, 0.0000, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.2277, 0.2277, 0.0000, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.2277, 0.2277, 0.2277, 0.0000, 0.2277, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.0000, 0.2277, 0.2277, 0.2277,
         0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277],
        [0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.2277, 0.0000, 0.227

In [18]:
prior_adj = model.causal_graph.adj_mat.clone().detach()
#scaled_prior = scale_tensor(prior_adj)
for i, row in enumerate(prior_adj):
    for j, column in enumerate(row):
        if (j in non_causal_indices) and (i in causal_indices) & (i!=j):
            continue
        else:
            prior_adj[i,j] = 0

In [19]:
pd.DataFrame(prior_adj.cpu().detach().numpy(),index=cols,columns=cols)

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
aimp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
amud,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
arnd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
asin1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
asin2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
adbr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
adfl,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
bed1,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227679,0.227679,0.227679
bed2,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227679,0.227679,0.227679
bfo1,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.227679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.227679,0.227679,0.227679


In [ ]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]
num_nodes = len(train_df.columns)
BATCH_SIZE = 50
device = torch.device('cpu')


new_metadata = []
edge_correct = 0
instantaneous_correct = 0
lagged_correct = 0
counterfactual_correct = 0 
rr_correct = 0
total_correct = 0
total_checked = 0
incorrect = []

for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    logger.info('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    #start_len = mod_df.shape[0]
    #if start_len >1000:
        #start_len = 1000
    start_len = 8
    start_time = anomaly_time- timedelta(seconds=start_len)
    finish_time = end_time + timedelta(seconds=start_len)
    mod_df = test_df[(test_df.index>= start_time) & (test_df.index<= finish_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]

    """
    FIND THE OPTIMAL NUMBER OF LAGS
    """
    TIME_STEPS = 8#max(most_frequent(find_optimal_lags_for_dataframe(mod_df))+1,3)

    dataset = TimeSeriesDataset(mod_df,device=device, time_steps=TIME_STEPS)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    X_data = torch.empty(0,device=device)
    T_data = torch.empty(0,device=device)
    for batch_idx, (X_batch, time_batch) in enumerate(dataloader):
        X_data = torch.cat((X_data[:batch_idx], X_batch, X_data[batch_idx:]))
        T_data = torch.cat((T_data[:batch_idx], time_batch, T_data[batch_idx:]))
    
    fine_tuned = CausalGraphVAE(input_dim=num_nodes, hidden_dim=hidden_dim,
                           latent_dim=latent_dim, num_nodes=num_nodes,device=device,
                           time_steps=TIME_STEPS,prior_adj=prior_adj.to(device),instantaneous_weight=0.5)
    optimizer = torch.optim.AdamW(fine_tuned.parameters(), lr=0.0001, weight_decay=1e-3)  # L2 Regularization

    
    fine_tuned.train_model(dataloader, optimizer, num_epochs=2500, patience=20,BATCH_SIZE=BATCH_SIZE,rho_max=2.0,alpha_max=2.0)

    causes = fine_tuned.infer_causal_effect(X_data.to(torch.float32).to(device),T_data.to(torch.float32).to(device),anomaly,cols,non_causal_indices=non_causal_indices)
    
    causes = causes.filter(items=potential_causes, axis=0)
    edge_cause_1 = causes.sort_values(by='causes',ascending=False)[0:3].index[0]
    edge_cause_2 = causes.sort_values(by='causes',ascending=False)[0:3].index[1]
    edge_cause_3 = causes.sort_values(by='causes',ascending=False)[0:3].index[2]
    
    instant_cause_1 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[0]
    instant_cause_2 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[1]
    instant_cause_3 = causes.sort_values(by='instantaneous',ascending=False)[0:3].index[2]
    
    lag_cause_1 = causes.sort_values(by='lagged',ascending=False)[0:3].index[0]
    lag_cause_2 = causes.sort_values(by='lagged',ascending=False)[0:3].index[1]
    lag_cause_3 = causes.sort_values(by='lagged',ascending=False)[0:3].index[2]
    
    counterfactual_cause_1 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[0]
    counterfactual_cause_2 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[1]
    counterfactual_cause_3 = causes.sort_values(by='counterfactuals',ascending=False)[0:3].index[2]

    rr_cause_1 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[0]
    rr_cause_2 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[1]
    rr_cause_3 = causes.sort_values(by='RootRank',ascending=False)[0:3].index[2]
    
    total_score_cause_1=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[0]
    total_score_cause_2=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[1]
    total_score_cause_3=causes.sort_values(by='causal_strength',ascending=False)[0:3].index[2]
    
    if root_cause == edge_cause_1:
        row['edge_cause_1'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_2'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_3'] = 1
    
    if root_cause == counterfactual_cause_1:
        row['counterfactual_cause_1'] = 1
    if root_cause == counterfactual_cause_2:
        row['counterfactual_cause_2'] = 1
    if root_cause == counterfactual_cause_3:
        row['counterfactual_cause_3'] = 1
    
    if root_cause == total_score_cause_1:
        row['total_score_cause_1'] = 1
    if root_cause == total_score_cause_2:
        row['total_score_cause_2'] = 1
    if root_cause == total_score_cause_3:
        row['total_score_cause_3'] = 1
    
    if root_cause == instant_cause_1:
        row['instant_cause_1'] = 1
    if root_cause == instant_cause_2:
        row['instant_cause_2'] = 1
    if root_cause == instant_cause_3:
        row['instant_cause_3'] = 1
    
    if root_cause == lag_cause_1:
        row['lag_cause_1'] = 1
    if root_cause == lag_cause_2:
        row['lag_cause_2'] = 1
    if root_cause == lag_cause_3:
        row['lag_cause_3'] = 1

    if root_cause == rr_cause_1:
        row['rr_cause_1'] = 1
    if root_cause == rr_cause_2:
        row['rr_cause_2'] = 1
    if root_cause == rr_cause_3:
        row['rr_cause_3'] = 1
    
    new_metadata.append(row)
    
    if root_cause in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3]:
        total_correct+=1
    if root_cause in [edge_cause_1 , edge_cause_2 , edge_cause_3]:
        edge_correct+=1
    if root_cause in [counterfactual_cause_1 , counterfactual_cause_2 , counterfactual_cause_3]:
        counterfactual_correct+=1
    if root_cause in [instant_cause_1 , instant_cause_2 , instant_cause_3]:
        instantaneous_correct+=1
    if root_cause in [lag_cause_1 , lag_cause_2 , lag_cause_3]:
        lagged_correct+=1
    if root_cause in [rr_cause_1 , rr_cause_2 , rr_cause_3]:
        rr_correct+=1
    
    total_accuracy = total_correct/total_checked* 100
    edge_accuracy = edge_correct/total_checked* 100
    cf_accuracy = counterfactual_correct/total_checked* 100
    instant_accuracy = instantaneous_correct/total_checked* 100
    lag_accuracy = lagged_correct/total_checked* 100
    rr_accuracy = rr_correct/total_checked* 100
    
    if root_cause not in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3,edge_cause_1 , edge_cause_2 , edge_cause_3 ]:
        incorrect.append(i) 
    logger.info(f"Edge Accuracy = {edge_accuracy:.2f}, Instantaneous Accuracy = {instant_accuracy:.2f}, Lagged Accuracy = {lag_accuracy:.2f}, Counterfactual Accuracy = {cf_accuracy:.2f},  Blended Accuracy = {total_accuracy:.2f},  RR Accuracy = {rr_accuracy:.2f}  ") 



2025-03-27 15:59:35,941 INFO -- Model: 0


Anomaly: cfo1
Epoch 1: Loss = 52125.9680
Recon Loss = 2567.2012, KL Loss = 0.0000, Lagrangian Loss = 54007.7461
Epoch 51: Loss = 27180.5833
Recon Loss = 180.6280, KL Loss = 0.0594, Lagrangian Loss = 29511.3438
Epoch 101: Loss = 13462.0625
Recon Loss = 99.3084, KL Loss = 0.1562, Lagrangian Loss = 14516.2598
Epoch 151: Loss = 5184.7607
Recon Loss = 105.9923, KL Loss = 0.2191, Lagrangian Loss = 5563.6338
Epoch 201: Loss = 2618.2565
Recon Loss = 63.0276, KL Loss = 0.3447, Lagrangian Loss = 2798.8047
Epoch 251: Loss = 1521.6651
Recon Loss = 61.2843, KL Loss = 0.4547, Lagrangian Loss = 1594.9978
Epoch 301: Loss = 971.8837
Recon Loss = 59.0003, KL Loss = 0.5661, Lagrangian Loss = 995.8981
Epoch 351: Loss = 659.2572
Recon Loss = 55.3028, KL Loss = 0.6419, Lagrangian Loss = 660.7437
Epoch 401: Loss = 472.5373
Recon Loss = 52.8800, KL Loss = 0.6873, Lagrangian Loss = 466.2163
Epoch 451: Loss = 346.8952
Recon Loss = 42.3673, KL Loss = 0.7178, Lagrangian Loss = 331.1893
Epoch 501: Loss = 266.3982


2025-03-27 18:27:26,672 INFO -- Edge Accuracy = 100.00, Instantaneous Accuracy = 100.00, Lagged Accuracy = 0.00, Counterfactual Accuracy = 0.00,  Blended Accuracy = 100.00,  RR Accuracy = 0.00  
2025-03-27 18:27:26,672 INFO -- Model: 1


Anomaly: cfo1
Epoch 1: Loss = 56903.4561
Recon Loss = 549.4895, KL Loss = 0.0000, Lagrangian Loss = 50390.7344
Epoch 51: Loss = 479.4792
Recon Loss = 48.0876, KL Loss = 0.0964, Lagrangian Loss = 413.0755
Epoch 101: Loss = 85.9704
Recon Loss = 42.6763, KL Loss = 0.2127, Lagrangian Loss = 46.8242
Epoch 151: Loss = 29.6169
Recon Loss = 16.6747, KL Loss = 0.3053, Lagrangian Loss = 14.7006
Epoch 201: Loss = 19.7449
Recon Loss = 7.5022, KL Loss = 0.3045, Lagrangian Loss = 10.9338
Epoch 251: Loss = 15.0762
Recon Loss = 4.5496, KL Loss = 0.2922, Lagrangian Loss = 10.7476
Epoch 301: Loss = 14.2192
Recon Loss = 3.6823, KL Loss = 0.2518, Lagrangian Loss = 10.8021
Epoch 351: Loss = 13.9130
Recon Loss = 3.9199, KL Loss = 0.2520, Lagrangian Loss = 10.8803
Early stopping triggered. Last Epoch: 398
Recon Loss = 2.0534, KL Loss = 0.1879, Lagrangian Loss = 10.9675


2025-03-28 01:02:37,938 INFO -- Edge Accuracy = 50.00, Instantaneous Accuracy = 50.00, Lagged Accuracy = 0.00, Counterfactual Accuracy = 50.00,  Blended Accuracy = 100.00,  RR Accuracy = 0.00  
2025-03-28 01:02:37,939 INFO -- Model: 2


...

Anomaly: cfo1
Epoch 1: Loss = 25480.3125
Recon Loss = 2223.8022, KL Loss = 0.0000, Lagrangian Loss = 27126.4414
Epoch 51: Loss = 15322.9607
Recon Loss = 291.0033, KL Loss = 0.0430, Lagrangian Loss = 17559.3125
Epoch 101: Loss = 9150.1963
Recon Loss = 162.7530, KL Loss = 0.1057, Lagrangian Loss = 10365.7695
Epoch 151: Loss = 5618.0132
Recon Loss = 118.6054, KL Loss = 0.1904, Lagrangian Loss = 6402.7637
Epoch 201: Loss = 3623.9147
Recon Loss = 91.5971, KL Loss = 0.2791, Lagrangian Loss = 4039.7654
Epoch 251: Loss = 2496.5702
Recon Loss = 79.9366, KL Loss = 0.3845, Lagrangian Loss = 2894.7932
Epoch 301: Loss = 1800.7126
Recon Loss = 73.6155, KL Loss = 0.5096, Lagrangian Loss = 2050.0212
Epoch 351: Loss = 1335.9071
Recon Loss = 57.2595, KL Loss = 0.5860, Lagrangian Loss = 1529.8342
Epoch 401: Loss = 1023.0784
Recon Loss = 62.2438, KL Loss = 0.7112, Lagrangian Loss = 1125.5670
Epoch 451: Loss = 803.4954
Recon Loss = 58.6227, KL Loss = 0.8200, Lagrangian Loss = 875.5508
Epoch 501: Loss = 640

2025-03-28 08:28:18,296 INFO -- Edge Accuracy = 60.00, Instantaneous Accuracy = 40.00, Lagged Accuracy = 60.00, Counterfactual Accuracy = 80.00,  Blended Accuracy = 100.00,  RR Accuracy = 60.00  
2025-03-28 08:28:18,297 INFO -- Model: 5


Anomaly: cfo1
Epoch 1: Loss = 40493.1500
Recon Loss = 2348.5393, KL Loss = 0.0000, Lagrangian Loss = 39699.7070
Epoch 51: Loss = 11206.3636
Recon Loss = 134.2639, KL Loss = 0.0513, Lagrangian Loss = 11746.1641
Epoch 101: Loss = 4107.0412
Recon Loss = 59.0447, KL Loss = 0.1301, Lagrangian Loss = 4296.3052
Epoch 151: Loss = 1686.3780
Recon Loss = 47.6314, KL Loss = 0.1466, Lagrangian Loss = 1747.9731
Epoch 201: Loss = 841.2046
Recon Loss = 45.8567, KL Loss = 0.2183, Lagrangian Loss = 850.1748
Epoch 251: Loss = 482.7141
Recon Loss = 42.4154, KL Loss = 0.3256, Lagrangian Loss = 471.8865
Epoch 301: Loss = 314.9741
Recon Loss = 31.9251, KL Loss = 0.4045, Lagrangian Loss = 294.0592
Epoch 351: Loss = 221.1140
Recon Loss = 47.2993, KL Loss = 0.4129, Lagrangian Loss = 196.0555
Epoch 401: Loss = 162.7727
Recon Loss = 32.7919, KL Loss = 0.4087, Lagrangian Loss = 139.1293
Epoch 451: Loss = 124.2073
Recon Loss = 28.7994, KL Loss = 0.4012, Lagrangian Loss = 100.3740
Epoch 501: Loss = 97.9532
Recon Lo

2025-03-28 11:10:49,471 INFO -- Edge Accuracy = 66.67, Instantaneous Accuracy = 50.00, Lagged Accuracy = 50.00, Counterfactual Accuracy = 66.67,  Blended Accuracy = 100.00,  RR Accuracy = 50.00  
2025-03-28 11:10:49,472 INFO -- Model: 6


Anomaly: cso1
Epoch 1: Loss = 37500.1094
Recon Loss = 2737.5425, KL Loss = 0.0000, Lagrangian Loss = 40637.1797
Epoch 51: Loss = 31147.4046
Recon Loss = 258.9304, KL Loss = 0.0484, Lagrangian Loss = 36042.4609
Epoch 101: Loss = 27147.9037
Recon Loss = 152.1610, KL Loss = 0.1264, Lagrangian Loss = 31456.7324
Epoch 151: Loss = 22384.7804
Recon Loss = 122.6846, KL Loss = 0.2073, Lagrangian Loss = 25928.9785
Epoch 201: Loss = 17066.4051
Recon Loss = 90.3979, KL Loss = 0.3293, Lagrangian Loss = 19804.0605
Epoch 251: Loss = 12522.3465
Recon Loss = 85.9347, KL Loss = 0.4476, Lagrangian Loss = 14393.8584
Epoch 301: Loss = 8623.7861
Recon Loss = 73.8880, KL Loss = 0.5582, Lagrangian Loss = 9799.8887
Epoch 351: Loss = 5974.3387
Recon Loss = 68.4404, KL Loss = 0.6780, Lagrangian Loss = 6908.8325
Epoch 401: Loss = 4231.7289
Recon Loss = 61.8500, KL Loss = 0.7681, Lagrangian Loss = 4831.4316
Epoch 451: Loss = 3140.3872
Recon Loss = 65.2844, KL Loss = 0.8569, Lagrangian Loss = 3656.6807
Epoch 501: L